In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

data_dir = '/content/drive/MyDrive/crop_part1'
img_size = 64


images = []
labels = []

for filename in os.listdir(data_dir):
    age = int(filename.split('_')[0])

    label = np.zeros(111)
    label[age] = 1.0
    label = np.exp(-(np.arange(111) - age)**2 / (2 * 9**2))
    label /= np.sum(label) 
    img = cv2.imread(os.path.join(data_dir, filename))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (img_size, img_size))
    images.append(img)
    labels.append(label)

images = np.array(images, dtype='float32') / 255.0
labels = np.array(labels, dtype='float32')


x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)





In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

input_shape = (img_size, img_size, 3)


model = tf.keras.models.Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(100, activation='softmax') 
])





In [10]:

def age_loss(y_true, y_pred):
    return tf.keras.losses.kullback_leibler_divergence(y_true, y_pred)

import tensorflow as tf

def age_accuracy(y_true, y_pred):
    mu_true = tf.reduce_sum(tf.range(100, dtype=tf.float32) * y_true, axis=1)
    mu_pred = tf.reduce_sum(tf.range(100, dtype=tf.float32) * y_pred, axis=1)
    acc = tf.reduce_mean(tf.cast(tf.equal(tf.round(mu_true), tf.round(mu_pred)), tf.float32))
    return acc

model.compile(optimizer='adam', loss=age_loss, metrics=[age_accuracy])


history = model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.2)







Epoch 1/10
98/98 [==============================] - 7s 27ms/step - loss: 0.4144 - age_accuracy: 0.0837 - val_loss: 0.4347 - val_age_accuracy: 0.0432
Epoch 2/10
98/98 [==============================] - 2s 17ms/step - loss: 0.4029 - age_accuracy: 0.0895 - val_loss: 0.3973 - val_age_accuracy: 0.0607
Epoch 3/10
98/98 [==============================] - 1s 12ms/step - loss: 0.3931 - age_accuracy: 0.0895 - val_loss: 0.4069 - val_age_accuracy: 0.0557
Epoch 4/10
98/98 [==============================] - 1s 11ms/step - loss: 0.3769 - age_accuracy: 0.0905 - val_loss: 0.3925 - val_age_accuracy: 0.0564
Epoch 5/10
98/98 [==============================] - 1s 10ms/step - loss: 0.3671 - age_accuracy: 0.0902 - val_loss: 0.3872 - val_age_accuracy: 0.0519
Epoch 6/10
98/98 [==============================] - 1s 10ms/step - loss: 0.3642 - age_accuracy: 0.0959 - val_loss: 0.4398 - val_age_accuracy: 0.0883
Epoch 7/10
98/98 [==============================] - 1s 10ms/step - loss: 0.3721 - age_accuracy: 0.0873 - v

In [11]:
y_pred = model.predict(x_test)
mu = np.sum(np.arange(100) * y_test, axis=1)
sigma2 = np.sum((np.arange(100) - mu.reshape(-1, 1))**2 * y_test, axis=1)
n = np.sum(np.arange(100) * y_pred, axis=1)
error = np.mean(1 - np.exp(- (n - mu)**2 / (2 * sigma2)))
print(f"Age estimation error: {error:.4f}")


62/62 [==============================] - 0s 3ms/step
Age estimation error: 0.2800


In [ ]:
###Briefly describe how the  ϵ  error behaves in the limit cases

###The ϵ error, also known as the training loss, is a measure of how well the model is able to fit the training data. 
###In the limit cases, the training loss can approach zero, which means the model perfectly fits the training data. However, 
##this does not necessarily mean that the model is able to generalize well to unseen data. On the other hand, if the training loss
##is very high, the model is not able to fit the training data well and is likely to perform poorly on both the training and test data.


##What can you say about the data sample?
#I have used the dataset of cropped images from UTK face dataset which consist on Face images from 1 to 111

##Further improvements?
#Some possible improvements to this model include:

##Increasing the size of the dataset by collecting more images and/or using data augmentation techniques.
##Trying different architectures such as deeper or wider networks to capture more complex features.
##Experimenting with different loss functions, such as mean squared error or mean absolute error.
##Tuning the hyperparameters of the model, such as the learning rate, batch size, number of epochs, etc.
##Investigating the use of transfer learning by fine-tuning a pre-trained model on a large dataset of faces.


##Is your model learning?
##Yes, I think the model is learning as the training loss is decreasing with each epoch.
##The validation loss is also decreasing, which indicates that the model is not overfitting to the training data.

